# Prepares train data for stage 1 binary classifer
## read all OK images, resize to (224,224)
## using pretrained model Xception to get 1024 features
## label as 0 
## read all NG images, resize to (224,224)
## using pretrained model DenseNet121 to get 1024 features
## label as 1
## Concatenate NG and ok Matrix to get final train_x (None, 1024) , and final_train_y (None, 1)

In [16]:
from keras.utils import to_categorical
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50
from keras.applications.densenet import DenseNet121, DenseNet201
from keras.preprocessing import image
#from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.preprocessing.image import ImageDataGenerator,  img_to_array, load_img

from keras.layers import Dense, GlobalAveragePooling2D

from keras.applications.xception import Xception, preprocess_input

#import cv2
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import subprocess
import os
import pickle
from keras.layers import Input, Conv2D, MaxPooling2D, merge, Lambda,UpSampling2D, concatenate, Reshape, Dropout,Cropping2D,Activation
from keras.models import Model, load_model
import pandas as pd
import sklearn 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.callbacks import Callback
from keras.applications.mobilenet import MobileNet
from keras.applications.nasnet import NASNetMobile,NASNetLarge
import matplotlib.image as mpimg

from xgboost.sklearn import XGBClassifier

import time


In [2]:

from tao_file_preprocess import make_ok_ng_dir 

In [3]:
data_path = './dataset/ARY_ADJ/'
ok_dir,ok_prefix,ng_dir,ng_prefix= make_ok_ng_dir(data_path)

['./dataset/ARY_ADJ/0/NG/AI', './dataset/ARY_ADJ/0/OK/AI', './dataset/ARY_ADJ/1/NG/AI', './dataset/ARY_ADJ/1/OK/AI', './dataset/ARY_ADJ/2/NG/AI', './dataset/ARY_ADJ/3/NG/AI', './dataset/ARY_ADJ/3/OK/AI', './dataset/ARY_ADJ/4/AI', './dataset/ARY_ADJ/5/AI', './dataset/ARY_ADJ/6/OK Natural/AI', './dataset/ARY_ADJ/6/OK Artificial/AI', './dataset/ARY_ADJ/7/NG/AI', './dataset/ARY_ADJ/7/OK/AI', './dataset/ARY_ADJ/8/OK/AI']
./dataset/ARY_ADJ/6/OK Artificial/AI 	 274
./dataset/ARY_ADJ/3/NG/AI 	 109
./dataset/ARY_ADJ/8/OK/AI 	 253
./dataset/ARY_ADJ/0/OK/AI 	 171
./dataset/ARY_ADJ/1/OK/AI 	 404
./dataset/ARY_ADJ/7/NG/AI 	 258
./dataset/ARY_ADJ/7/OK/AI 	 418
./dataset/ARY_ADJ/0/NG/AI 	 192
./dataset/ARY_ADJ/6/OK Natural/AI 	 135
./dataset/ARY_ADJ/1/NG/AI 	 423
./dataset/ARY_ADJ/4/AI 	 311
./dataset/ARY_ADJ/2/NG/AI 	 173
./dataset/ARY_ADJ/5/AI 	 409
./dataset/ARY_ADJ/3/OK/AI 	 301
['./dataset/ARY_ADJ/0/OK/AI', './dataset/ARY_ADJ/1/OK/AI', './dataset/ARY_ADJ/3/OK/AI', './dataset/ARY_ADJ/6/OK Natural

In [4]:
with open('final_dir_files.pkl','rb') as handle:
    
    final_dir_files = pickle.load(handle)

In [5]:
if not os.path.exists('./np_data_model/'): 
    os.makedirs('./np_data_model/')

In [6]:
save_dir = './np_data_model/'

In [17]:

def build_model():
    inputs = Input(shape=(384,512,3))
    #inputs = Lambda(lambda x: x/275. - 1.0)(inputs)
    # create the base pre-trained model
    base_model = DenseNet201(weights='imagenet',input_tensor=inputs,  include_top=True)
    
    x = base_model.output
    print (x.get_shape)
    model = Model(inputs=base_model.input, outputs=x)
    model.summary()
    
    return model

model_1 = build_model()
# compile the model (should be done *after* setting layers to non-trainable)
model_1.compile(optimizer='Nadam', loss='categorical_crossentropy',metrics=['accuracy'])

def first_prediction(model,data):
    pred_temp = model.predict(data)
    return pred_temp

82526208/82524592 [==============================] - 365s 4us/step
<bound method Tensor.get_shape of <tf.Tensor 'fc1000_2/Softmax:0' shape=(?, 1000) dtype=float32>>
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 384, 512, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 390, 518, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 192, 256, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 192,

In [18]:

def make_train_data(dirs,finale_dir_files):
    
    final_array =[]

   
    s = time.time()
    for i, dir  in enumerate(dirs):
        img_list=[]
        print ('current dir=', dir)
        
        for im in final_dir_files[dir]:
            s1 =s


            img = load_img(dir+'/'+im, target_size=(384, 512))  # this is a PIL image
            #img = load_img(dir+'/'+im)  # this is a PIL image
            x = img_to_array(img)
            x = preprocess_input(x)
            #print (x.shape)
            #x = cv2.cvtColor(y, cv2.COLOR_BGR2RGB)
            #x = x/127.5 -1.
            img_list.append(x)
            #print ('img_list=', len(img_list))
        testing_data = np.stack(img_list,axis=0)
        pred = first_prediction(model_1,testing_data)
        final_array.append(pred)
        e = time.time()    
        
        print ('batch {}, shape {},time = {}'.format(i,pred.shape,round(e -s1 ,4)))
        s1=e
 
    final_array = np.concatenate(final_array,axis=0)
    print ('final_array shape',final_array.shape)


    e = time.time()
    print ('time=', round(e -s ,4))
    
    return final_array


In [19]:
ok_train= make_train_data(ok_dir,final_dir_files)
np.save(save_dir +'train_x_ok_DenseNet201_1000'+'.npy',ok_train)
ok_label = np.zeros((ok_train.shape[0],1))
print ('ok image shape=', ok_train.shape, 'ok_label = ', ok_label.shape, 'ok image data completed')

current dir= ./dataset/ARY_ADJ/0/OK/AI
batch 0, shape (170, 1000),time = 10.7093
current dir= ./dataset/ARY_ADJ/1/OK/AI
batch 1, shape (403, 1000),time = 23.7978
current dir= ./dataset/ARY_ADJ/3/OK/AI
batch 2, shape (300, 1000),time = 33.6492
current dir= ./dataset/ARY_ADJ/6/OK Natural/AI
batch 3, shape (134, 1000),time = 38.4331
current dir= ./dataset/ARY_ADJ/6/OK Artificial/AI
batch 4, shape (273, 1000),time = 46.9413
current dir= ./dataset/ARY_ADJ/7/OK/AI
batch 5, shape (418, 1000),time = 60.1745
current dir= ./dataset/ARY_ADJ/8/OK/AI
batch 6, shape (252, 1000),time = 68.9865
final_array shape (1950, 1000)
time= 68.9897
ok image shape= (1950, 1000) ok_label =  (1950, 1) ok image data completed


In [20]:
ng_train= make_train_data(ng_dir,final_dir_files)
np.save(save_dir +'train_x_ng_DenseNet201_1000'+'.npy',ng_train)
ng_label = np.ones((ng_train.shape[0],1))
print ('ng image shape=', ng_train.shape, 'ng_label = ', ng_label.shape, 'ng image data completed')

current dir= ./dataset/ARY_ADJ/0/NG/AI
batch 0, shape (191, 1000),time = 6.9204
current dir= ./dataset/ARY_ADJ/1/NG/AI
batch 1, shape (422, 1000),time = 19.4281
current dir= ./dataset/ARY_ADJ/2/NG/AI
batch 2, shape (172, 1000),time = 24.6097
current dir= ./dataset/ARY_ADJ/3/NG/AI
batch 3, shape (108, 1000),time = 27.7959
current dir= ./dataset/ARY_ADJ/4/AI
batch 4, shape (310, 1000),time = 38.1204
current dir= ./dataset/ARY_ADJ/5/AI
batch 5, shape (408, 1000),time = 51.2932
current dir= ./dataset/ARY_ADJ/7/NG/AI
batch 6, shape (258, 1000),time = 58.9608
final_array shape (1869, 1000)
time= 58.9617
ng image shape= (1869, 1000) ng_label =  (1869, 1) ng image data completed


In [21]:
final_train_x = np.concatenate([ok_train,ng_train],axis=0)
np.save(save_dir+'final_train_x_DenseNet201_1000.npy',final_train_x)
print ('final_train_x_Xception1000 shape', final_train_x.shape, 'saved')
final_train_y = np.concatenate([ok_label,ng_label],axis=0)

np.save(save_dir+'final_train_y_DenseNet201_1000.npy',final_train_y)
print ('final_train_y_Xception1000 shape', final_train_y.shape, 'saved')

final_train_x_Xception1000 shape (3819, 1000) saved
final_train_y_Xception1000 shape (3819, 1) saved


In [23]:
import datetime
print (datetime.datetime.now())

2018-11-13 15:18:43.532946
